In [2]:
import praw
from praw.models import MoreComments
import pandas as pd
from datetime import datetime
from praw.exceptions import RedditAPIException

In [3]:
reddit = praw.Reddit(
)


In [4]:
df = pd.read_csv('labeled-data.csv',index_col=0)
positives = set(open('positive.txt').read().split())
negatives = set(open('negative.txt').read().split())

df

,$MSFT,$AAPL,$TSLA,$AMZN,$NFLX,$META,$NVDA,$AMD
Guysmarket,"['w69hk8', 'vaoptj', 'w91vth', 'vaevci']","['137z7xg', 'x6oms2', 'vfgi4j', 'z68fnt', 'vfy...","['12s8cfb', 'vaevci']","['vaoptj', 'vaevci']",['w3cwti'],NaN,"['13ne5be', 'yx6rl6', 'yx0gth']",['wexhxz']
predictany007,"['13igt6u', 'w9cih5', '12zedri', '13j5014', '1...","['y83owq', '13rl2eq', '136w8h8', '138ikrk', 'z...","['13i691q', '12rq3pf', '12svh8b', '120qykr', '...","['13qudsg', '13rl2eq', 'z122gg', 'zya1j1', 'yv...","['12rqd26', '13pwxdb', '13l69lg', '10gdagf', '...","['13ppjis', 'yldmuu', 'v44p5o']",NaN,"['xy1yx2', 'weh5zm']"
WickedSensitiveCrew,"['13ia8cg', '1154egt', 'zrouhg', 'zvzi00', 'v8...","['12qwfjn', 'zvzi00', '10jckme', 'zbtscx', 'yd...",['12ekzkx'],"['zvzi00', 'zbq9l8']",NaN,['zvzi00'],NaN,NaN
gaurav0792,"['10kk8x1', '11fvirg']",['11fvirg'],NaN,NaN,['zmpx3s'],['116o2zl'],NaN,NaN
DaddyDersch,"['12yvp9l', '12yvqq0', '10kh5rn']","['xsd18z', 'yqwe59', 'xqrdms', 'xouv0s', 'yjkj...","['12r45cv', '12s9361', '1144ms6', '11a9amh', '...",NaN,"['12r45cv', '12r46qm']","['12zyzqv', '12zzagw']","['13r0uu2', '13r0wga']","['vz5ejc', 'vzz7j9', 'w29gw7', 'w29flo', 'vz5d..."
msaleem,"['10s88q8', '10tv0xs', '10t0lwx']","['12buzi8', '10hueqy', '10lvspw', '10tv0xs', '...",NaN,"['11xfwwa', '10hueqy', '1035yg6', '10tv0xs', '...","['10juhvh', '1097x21']","['13m5yke', '11djv5g', '11f7pih', '1027i7o', '...",NaN,['10root9']
bwang29,['yf8b1n'],['yf8b1n'],NaN,NaN,['yf8b1n'],['yf8b1n'],NaN,['yf8b1n']
NoobInvester018,['137updu'],['139xgp2'],NaN,NaN,NaN,NaN,['13mfz75'],['137updu']
polloponzi,['v6dbh4'],['v6dbh4'],NaN,['z3hx10'],NaN,NaN,['13r6dna'],NaN
dennis77,['wikghr'],['wq010n'],['wikghr'],NaN,NaN,NaN,NaN,['wikghr']


In [5]:
def post_sentiment(text):
    
    p = len([w for w in text if w in positives])
    n = len([w for w in text if w in negatives])
    
    if n == 0:
        return 0.0
    
    return round(p / n, 3)

In [6]:
post = reddit.submission(id='w69hk8')
title = post.title
body = post.selftext

In [17]:
def post_to_dict(post_id):
    
    post = reddit.submission(id=post_id)
    date = datetime.fromtimestamp(post.created_utc)
    text = (post.title + post.selftext).split()
    sentiment = post_sentiment(text)
    post_dict = {'Title' : post.title, 'Author' : post.author, 'Subreddit' : post.subreddit, 'Date': date, 'Sentiment' : sentiment}
    return post_dict

In [18]:
post_to_dict('108b31n')

{'Title': 'Microsoft reportedly plans to invest $10 billion in creator of buzzy A.I. tool ChatGPT',
 'Author': Redditor(name='_hiddenscout'),
 'Subreddit': Subreddit(display_name='stocks'),
 'Date': datetime.datetime(2023, 1, 10, 9, 25, 27),
 'Sentiment': 2.333}